# Lab Vision Sytems: Session 3

# Today:

### 1: Solution Assignment 1
### 2: Learning and Optimization
### 3: Multi-Layer Perceptron (MLP)
### 4: Convolutional Neural Networks (CNN)

#### Colab tips:
    - https://medium.com/@oribarel/getting-the-most-out-of-your-google-colab-2b0585f82403
    - https://medium.com/datadriveninvestor/speed-up-your-image-training-on-google-colab-dc95ea1491cf


# 1: Solution Assignment 1

By Salih Marangoz and Elif Cansu Yildiz 

# 2: Learning and Optimization

# 3: Multi-Layer Perceptron

# 4: Convolutional Networks (CNNs)

In [ ]:
import os
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm

plt.style.use('seaborn')

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets

## Data

In [ ]:
# Downloading and Loading Dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(),download=True)
 
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [ ]:
# Fitting data loaders for iterating
B_SIZE = 256

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=B_SIZE, 
                                           shuffle=True) 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=B_SIZE, 
                                          shuffle=False)

## Model

In [ ]:
class CNN(nn.Module):
    """ 
    Varation of LeNet: a simple CNN model
    for handwritten digit recognition
    """
    def __init__(self):
        """ Model initializer """
        super().__init__()
        
        # layer 1
        conv1 = nn.Conv2d(in_channels=1, out_channels=16,  kernel_size=5, stride=1, padding=0)
        relu1 = nn.ReLU()
        maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.layer1 = nn.Sequential(
                conv1, relu1, maxpool1
            )
      
        # layer 2
        conv2 = nn.Conv2d(in_channels=16, out_channels=32,  kernel_size=5, stride=1, padding=0)
        relu2 = nn.ReLU()
        maxpool2 = nn.MaxPool2d(kernel_size=2)
        self.layer2 = nn.Sequential(
                conv2, relu2, maxpool2
            )
        
        # fully connected classifier
        in_dim = 32 * 4 * 4
        self.fc = nn.Linear(in_features=in_dim, out_features=10)
        
        return
        
    def forward(self, x):
        """ Forward pass """
        cur_b_size = x.shape[0]
        out1 = self.layer1(x)
        out2 = self.layer2(out1)
        out2_flat = out2.view(cur_b_size, -1)
        y = self.fc(out2_flat)
        return y
    
def count_model_params(model):
    """ Counting the number of learnable parameters in a nn.Module """
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return num_params

In [ ]:
cnn = CNN()
params = count_model_params(cnn)
print(f"Model has {params} learnable parameters")

In [ ]:
cnn

## Training

In [ ]:
LR = 3e-4
EPOCHS = 100
EVAL_FREQ = 1
SAVE_FREQ = 10

In [ ]:
stats = {
    "epoch": [],
    "train_loss": [],
    "valid_loss": [],
    "accuracy": []
}
init_epoch = 0

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cnn = cnn.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(params=cnn.parameters(), lr=LR)

In [ ]:
@torch.no_grad()
def eval_model(model):
    """ Computing model accuracy """
    correct = 0
    total = 0
    loss_list = []
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass only to get logits/output
        outputs = model(images)
                 
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
            
        # Get predictions from the maximum value
        preds = torch.argmax(outputs, dim=1)
        correct += len( torch.where(preds==labels)[0] )
        total += len(labels)
                 
    # Total correct predictions and loss
    accuracy = correct / total * 100
    loss = np.mean(loss_list)
    return accuracy, loss


def save_model(model, optimizer, epoch, stats):
    """ Saving model checkpoint """
    
    if(not os.path.exists("models")):
        os.makedirs("models")
    savepath = f"models/checkpoint_epoch_{epoch}.pth"

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'stats': stats
    }, savepath)
    return


def load_model(model, optimizer, savepath):
    """ Loading pretrained checkpoint """
    
    checkpoint = torch.load(savepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint["epoch"]
    stats = checkpoint["stats"]
    
    return model, optimizer, epoch, stats

In [ ]:
# loading model 
savepath = os.path.join(os.getcwd(), "models", "checkpoint_epoch_70.pth")
model, optimizer, init_epoch, stats = load_model(cnn, optimizer, savepath)

In [ ]:
loss_hist = []

for epoch in range(init_epoch, EPOCHS):
    loss_list = []
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, (images, labels) in progress_bar:
        images = images.to(device)
        labels = labels.to(device)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = cnn(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
        progress_bar.set_description(f"Epoch {epoch+1} Iter {i+1}: loss {loss.item():.5f}. ")
             
    loss_hist.append(np.mean(loss_list))
    stats["epoch"].append(epoch)
    stats["train_loss"].append(loss_hist[-1])
    
    # evaluating model
    if epoch % EVAL_FREQ == 0:
        accuracy, valid_loss = eval_model(cnn)  
        print(f"Accuracy at epoch {epoch}: {round(accuracy, 2)}%")
    else:   
        accuracy, valid_loss = -1, -1
    stats["accuracy"].append(accuracy)
    stats["valid_loss"].append(valid_loss)
    
    # saving checkpoint
    if epoch % SAVE_FREQ == 0:
        save_model(model=cnn, optimizer=optimizer, epoch=epoch, stats=stats)

In [ ]:
accuracy, _ = eval_model(cnn)  
print(f"Classification accuracy: {round(accuracy, 2)}%")

In [ ]:
epochs = np.array(stats["epoch"])
train_loss = np.array(stats["train_loss"])

eval_loss = np.array(stats["valid_loss"])
accuracy = np.array(stats["accuracy"])
eval_idx = np.where(eval_loss != -1)[0]

In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_size_inches(16,5)

for a in ax:

    a.plot(epochs+1, train_loss, label="Train Loss", linewidth=3)
    a.plot(epochs[eval_idx]+1, eval_loss[eval_idx], c="red", label="Eval Loss", linewidth=3)
#     a.scatter(epochs[eval_idx]+1, eval_loss[eval_idx], c="red", s=100, marker="x")
    a.legend(loc="best")
    a.set_xlabel("Epochs")
    a.set_ylabel("CE Loss value")

ax[0].set_title("Training-Eval Progress")
ax[1].set_title("Training-Eval Progress (Log)")
ax[1].set_yscale("log")

plt.show()

In [ ]:
def smooth(f, K=5):
    """ Smoothing a function using a low-pass filter (mean) of size K """
    kernel = np.ones(K) / K
    f = np.concatenate([f[:int(K//2)], f, f[int(-K//2):]])  # to account for boundaries
    smooth_f = np.convolve(f, kernel, mode="same")
    smooth_f = smooth_f[K//2: -K//2]  # removing boundary-fixes
    return smooth_f

In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_size_inches(16,5)

ax[0].plot(epochs[eval_idx]+1, accuracy[eval_idx], c="red", label="Accuracy", linewidth=3)
ax[0].legend(loc="best")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Classification Accuracy")
ax[0].set_title("Eval Accuracy Progress")

zoomed = accuracy[10:]
filtered = smooth(zoomed, K=9)

ax[1].plot(epochs[10:]+1, accuracy[10:], c="red", label="Accuracy", linewidth=3)
ax[1].plot(epochs[10:]+1, filtered, c="blue", label="Smoothed Accuracy", linewidth=3)
ax[1].legend(loc="best")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Classification Accuracy")
ax[1].set_title("Eval Accuracy Progress Focused on Flat Area")

plt.show()

# Assignment 3
- Train and compare the MLP from Assignment 2 and a simple CNN on the SVHN dataset (available in PyTorch) with optimized hyper-parameters
- Visualize several convolutional kernels and their activations
- Train CNNs with L1, L2, Elastic regularization and No-Regularization. Which method achieves the best results?
compare the model performance. With which regularization do you obtain the best results?
- Train CNNs with and without Dropout. Compare the results: accuracy, training time, number of parameters, ...
- Extra Point:
  - Train CNNs with the following Pooling methods: MaxPooling, AvgPooling or a combination of both and compare the results

#### **Due Date**: Sunday 30th May at 23:59
#### Submit it by mail using the subject: **CudaLab: Assignment3**
####  Send me the following: Jupyter Notebook after running, Jupyter export as html, any other .py files or images used.

# References
 - https://www.deeplearningbook.org/
 - http://cs231n.stanford.edu/
 - https://towardsdatascience.com/all-you-want-to-know-about-deep-learning-8d68dcffc258
 - https://machinelearningmastery.com/learning-curves-for-diagnosing-machine-learning-model-performance/
 - https://github.com/vdumoulin/conv_arithmetic
 

<div class=alert style="background-color:#F5F5F5; border-color:#C8C8C8">
    <b>Angel Villar-Corrales</b><br>
    <ul>
       <li> <b>Email</b>: villar@ais.uni-bonn.de
       <li> <b>Website</b>: angelvillarcorrales.com
    </ul>
</div> 